# Dispatching Scenario: Two types of material and destinations

This notebook model and simulate the dispatching scenario including:

- Hauling Cycle: Drive empty-load-drive full-unload
- Queuing at source and destination
- Transporting of two types of material (Ore and Waste) with more than 2 trucks to two destinations (Stockpile and Dump)
- Shift Amount till the source location is empty or destinations are full

In [ ]:
import datetime, time
import simpy

import numpy as np
import pandas as pd
import shapely.geometry

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

In [ ]:
NR_TRUCKS = 3 # Number of trucks that transport Ore material
NR_DUMP_TRUCKS = 3 # Number of trucks that transport Waste material
TOTAL_AMOUNT_ORE = 100 # Total amount of Ore material at Source in tons
TOTAL_AMOUNT_WASTE = 500 # Total amount of Waste material at Source in tons
TRUCK_PAYLOAD = 20 # Truck's capacity in tons

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

In [ ]:
# create a Site object based on desired mixin classes
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasMultiContainer,
        core.HasResource,
    ),
    {},
)

# create a TransportProcessingResource object based on desired mixin classes
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.MultiContainerDependentMovable,
        core.HasResource,
        core.Processor,
        core.Identifiable,
        core.Log,
    ),
    {},
)

In [ ]:
ocation_source = shapely.geometry.Point(4.18055556, 52.18664444)
data_source = {"env": my_env,
                  "name": "source",
                  "geometry": location_source,
                  "store_capacity": 4,
                  "nr_resources": 1,
                  "initials": [
                    {
                        "id": "ore", "level": TOTAL_AMOUNT_ORE, "capacity": TOTAL_AMOUNT_ORE,
                        "id": "waste", "level": TOTAL_AMOUNT_WASTE, "capacity": TOTAL_AMOUNT_WASTE,
                    },
                  ],
                 }
source = Site(**data_source)

In [ ]:
location_stokpile = shapely.geometry.Point(4.25222222, 52.11428333)
data_stokpile = {"env": my_env,
                "name": "stokpile",
                "geometry": location_stokpile,
                "store_capacity": 2,
                "nr_resources": 1,
                "initials": [
                    {
                        "id": "ore", "level": 0, "capacity": TOTAL_AMOUNT_ORE,
                    },
                ],
                 }
stokpile = Site(**data_stokpile)

location_dump = shapely.geometry.Point(42.25222222, 78.11428333)
data_dump = {"env": my_env,
                "name": "dump",
                "geometry": location_dump,
                "store_capacity": 1,
                "nr_resources": 1,
                "initials": [
                    {
                        "id": "waste", "level": 0, "capacity": TOTAL_AMOUNT_WASTE,
                    },
                ],
                 }
dump = Site(**data_dump)

In [ ]:
trucks = {}
for i in range(NR_TRUCKS):
    trucks[f"truck {i}"] = TransportProcessingResource(
        env=my_env,
        name=f"truck{i}",
        geometry=location_stokpile, 
        store_capacity=4,
        nr_resources=1,
        compute_v=lambda x: 10,
        initials=[
            {"id": "ore", "level": 0, "capacity": TRUCK_PAYLOAD},
         ],
    )

dump_trucks = {}
for i in range(NR_DUMP_TRUCKS):
    dump_trucks[f"dump_truck {i}"] = TransportProcessingResource(
        env=my_env,
        name=f"dump_truck{i}",
        geometry=location_dump, 
        store_capacity=4,
        nr_resources=1,
        compute_v=lambda x: 10,
        initials=[
            {"id": "waste", "level": 0, "capacity": TRUCK_PAYLOAD},
         ],
    )

In [ ]:
processes = []
for i in range(NR_TRUCKS):
    truck = trucks[f"truck {i}"]
    requested_resources={}
    
    processes.append(
        model.WhileActivity(
            env=my_env,
            name=f"while {truck.name}",
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name=f"sequence {truck.name}",
                    registry=registry,
                    sub_processes=[
                        model.MoveActivity(
                            env=my_env,
                            name=f"driving empty {truck.name}",
                            registry=registry,
                            mover=truck,
                            destination=source,
                            duration=10,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name=f"load ore {truck.name}",
                            registry=registry,
                            processor=truck,
                            origin=source,
                            destination=truck,
                            amount=TRUCK_PAYLOAD,
                            duration=10,
                            id_="ore",
                            requested_resources=requested_resources,
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name=f"driving full {truck.name}",
                            registry=registry,
                            mover=truck,
                            destination=stokpile,
                            duration=10,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name=f"unload ore{truck.name}",
                            registry=registry,
                            processor=truck,
                            origin=truck,
                            destination=stokpile,
                            amount=TRUCK_PAYLOAD,
                            duration=10,
                            id_="ore",
                            requested_resources=requested_resources,
                        ),
                    ]
                )
            ],
            condition_event={
                "type": "ore", "concept": stokpile, "state": "full", "id_":"default_reservations"
            },
        ),
    )

In [ ]:
for i in range(NR_DUMP_TRUCKS):
    dump_truck = dump_trucks[f"dump_truck {i}"]
    requested_resources={}
    
    processes.append(
        model.WhileActivity(
            env=my_env,
            name=f"while {dump_truck.name}",
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name=f"sequence {dump_truck.name}",
                    registry=registry,
                    sub_processes=[
                        model.MoveActivity(
                            env=my_env,
                            name=f"driving empty {dump_truck.name}",
                            registry=registry,
                            mover=dump_truck,
                            destination=source,
                            duration=10,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name=f"load waste {dump_truck.name}",
                            registry=registry,
                            processor=dump_truck,
                            origin=source,
                            destination=dump_truck,
                            amount=TRUCK_PAYLOAD,
                            duration=10,
                            id_="waste",
                            requested_resources=requested_resources,
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name=f"driving full {dump_truck.name}",
                            registry=registry,
                            mover=dump_truck,
                            destination=dump,
                            duration=10,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name=f"unload waste{dump_truck.name}",
                            registry=registry,
                            processor=dump_truck,
                            origin=dump_truck,
                            destination=dump,
                            amount=TRUCK_PAYLOAD,
                            duration=10,
                            id_="waste",
                            requested_resources=requested_resources,
                        ),
                    ]
                )
            ],
            condition_event={
                "type": "waste", "concept": dump, "state": "full", "id_":"default_reservations"
            },
        ),
    )


In [ ]:
model.register_processes([*processes])
my_env.run()